# 제3장 협업 필터링 추천 시스템

**협업 필터링(CF)** : 사용자를 성별이나 직업과 같은 인구 통계적 변수를 기준으로 나누지 않고 취향을 기준으로 나누는 것이다

## 3.1 협업 필터링의 원리

추천의 대상이 되는 어떤 한 사람이 있으면, 이 사람과 취향이 비슷한 사람들(neighbor)을 찾아내기만 하면 이사람들이 공통적으로 좋아하는 제품,서비스를 추천 대상인 사람에게 추천하면 된다는 아이디어

## 3.2 유사도 지표

### 1) 상관계수
평가 자료가 연속값인 경우에 가장 이해하기 쉬운 유사도로는 상관계수(correlation coefficient)가 있다.  
일반적으로 상관계수를 말하면 **피어슨 상관계수(pearson correlation coefficient)** 를 말함

$r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^{n} (y_i - \bar{y})^2}}$

- $x,y$ : 두 사용자가 공통으로 평가한 아이템($I$) 중에서 $i$번째 아이템에 대한 이 두 사용자의 평가값이다.
- $\bar{x},\bar{y}$ : 각각의 평균값
- 상관계수는 최소 -1(완전 반대)에서 최대 1(완전 일치)까지의 값을 가진다.

### 2) 코사인 유사도
코사인 유사도에서는 각 아이템을 하나의 차원(dimension)으로 보고 사용자의 평가값을 좌표값으로 본다.
그렇게 되면 각 사용자의 평가값을 벡터로 해서 두사용자 간의 벡터의 각도(코사인값)를 구할 수 있다.

$\text{simil}(x, y) = \frac{\vec{x} \cdot \vec{y}}{\|\vec{x}\| \|\vec{y}\|} = \frac{\sum_{i=1}^{n} x_i y_i}{\sqrt{\sum_{i=1}^{n} x_i^2} \sqrt{\sum_{i=1}^{n} y_i^2}}$

만일 데이터가 이진값(binary)을 가진다면 위의 상관계수나 코사인 유사도는 사용할 수 없다  
이 경우 **타니모토 계수(Tanimoto coefficient)** 가 있다  

### 3) 타니모토 계수

$\text{simil}(x, y) = \frac{c}{a+b-c}$
- a: 사용자 x가 1의 값을 갖는 아이템의 수
- b: 사용자 y가 1의 값을 갖는 아이템의 수
- c: 사용자 x와 y가 공통적으로 1의 값을 갖는 아이템의 수

### 4) 자카드 계수(Jaccard coefficient)

이진수로 이루어진 데이터에 대해서 유사도를 구한다  
0~1 사이의 값을 가지며 두 집합이 동일하면 1의 값을 가지고, 공통  원소가 없으면 0을 가진다.  

$J(A, B) = \frac{|A \cap B|}{|A \cup B|}$
- $|A \cap B|$ : 두집합 $A$와 $B$의 교집합 크기
- $|A \cup B|$ : 두집합 $A$와 $B$의 합집합 크기


## 3.3 기본 CF 알고리즘  

1. 모든 사용자 간의 평가의 유사도를 계산한다. 위에서 설명한 상관계수, 코사인 유사도 등을 사용  

2. 현재 추천 대상이 되는 사람과 다른 사용자의 유사도를 추출한다.

3. 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가 값을 구한다
    - 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자와의 유사도로 가중해서 평균을 낸다.

4. 아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천한다.

In [22]:
from inspect import mod_dict

# 사전 준비
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [38]:
# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0) # 계산할때 NaN값이 있으면 오류 발생
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,index=rating_matrix.index,columns=rating_matrix.index)

In [7]:
user_similarity[1].sum()

174.35828033263857

In [8]:
# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id,movie_id):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        # 주어진 영화에 대해서 평가를 하지 않은 사용자의 위치확인 -> 이 사용자들에 대해서는 가중평균 계산에서 빼기 위함
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.dropna()
        # 가중평균 계산에서 빼기
        sim_scores = sim_scores.drop(none_rating_idx)
        # 가중합을 유사도의 합으로 나누어, 유사도를 기준으로 조정된 평균값을 계산
        # 가중합 : 사용자의 평점에 해당 사용자의 유사도를 곱한 값을 모두 더한 것
        mean_rating = np.dot(sim_scores,movie_ratings) / sim_scores.sum() # np.dot() 행렬곱
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple)

1.0201466389317266

### 연습문제

3-1 위의 코드를 수정해서 코사인 유사도 대신에 피어슨 상관계수를 사용하는 코드를 작성하고 RMSE를 계산하시오

In [36]:
user_similarity_pearson = np.corrcoef(matrix_dummy) # np.corrcoef() 피어슨 상관계수 계산 기본적으로 행을 기준으로 계산
user_similarity_pearson = pd.DataFrame(user_similarity_pearson,index=rating_matrix.index,columns=rating_matrix.index)

In [37]:
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity_pearson)
def CF_simple_pearson(user_id,movie_id):
    if movie_id in rating_matrix:
        sim_scores = user_similarity_pearson[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        # 주어진 영화에 대해서 평가를 하지 않은 사용자의 위치확인 -> 이 사용자들에 대해서는 가중평균 계산에서 빼기 위함
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.dropna()
        # 가중평균 계산에서 빼기
        sim_scores = sim_scores.drop(none_rating_idx)
        # 가중합을 유사도의 합으로 나누어, 유사도를 기준으로 조정된 평균값을 계산
        # 가중합 : 사용자의 평점에 해당 사용자의 유사도를 곱한 값을 모두 더한 것
        mean_rating = np.dot(sim_scores,movie_ratings) / sim_scores.sum() # np.dot() 행렬곱
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple_pearson)

1.1090429998697189

## 3.4 이웃을 고려한 CF

**K Nearest Neighbors(KNN)** : 이웃의 크기를 정해놓고 추천 대상 사용자와 가장 유사한 k명을 선택

<br>

**Thresholding** : 크기 대신 유사도의 기준(ex 상관계수 0.8 이상)을 정해 놓고 이 기준을 충족시키는 사용자를 이웃으로 정함  


In [39]:
# 모델별 RMSE를 계산하는 함수 
def score(model,neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie,neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# Neighbor size를 정해서 예측치를 계산하는 함수
def cf_knn(user_id,movie_id,neighbor_size=0):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
    
        # Neighbor size가 지정되지 않은 경우
        if neighbor_size ==0:
            mean_rating = np.dot(sim_scores,movie_ratings) / sim_scores.sum()
        else :
            # 해당 영화를 평가한 다른 사용자 수가 매우 작으면 계산에 문제가 생기기 때문에 평가자 수가 최소 2명 이상인 경우만 계산
            if len(sim_scores)>1:
                neighbor_size = min(neighbor_size,len(sim_scores))
                # 아래의 argsort()를 사용하기 위해서 numpy array로 바꿔준다
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 다른 사용자를 유사도 순서대로 정렬
                user_idx = np.argsort(sim_scores) # 배열을 오름차순으로 정렬한 후, 그에 해당하는 원래 인덱스를 반환
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                mean_rating = np.dot(sim_scores,movie_ratings) / sim_scores.sum()
            else :
                mean_rating = 3.0
    else :
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(cf_knn,neighbor_size=30)

1.0102247779961107

In [40]:
# 주어진 사용자에 대해 추천받기
# 전체 데이터로 full matrix와 cosine similarity 구하기
rating_matrix = ratings.pivot_table(values='rating',index='user_id',columns='movie_id')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,index = rating_matrix.index,columns=rating_matrix.index)

def recommender(user,n_items=10,neighbor_size=20):
    predictions = []
    # 현 사용자가 이미 평가한 인덱스 저장(추천에서 제외 하기 위해)
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user]>0].index
    items = rating_matrix.loc[user].drop(rated_index)
    for item in items.index:
        predictions.append(cf_knn(user,item,neighbor_size))
    recommendations = pd.Series(data=predictions,index = items.index,dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]
    recommend_items = movies.loc[recommendations.index]['title']
    return recommend_items

recommender(user=2,n_items=5,neighbor_size=30)

movie_id
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
1467                                     Cure, The (1995)
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

## 3.5 최적의 이웃 크기 결정

이웃의 크기 너무 클 때 : 집단별 취향의 차이가 없어지고 best-seller 방식과 크게 다를 바가 없게 된다.  
이웃의 크기 너무 작을 때 : 현재 사용자와 취향의 유사도가 매우 높은 소숭의 이웃의 평가만을 사용
- 기계학습의 과적합(over-fittign) 문제

<br>

구체적으로 얼마가 최적의 크기인지는 분야(domain)에 따라 차이가 있음 -> 최적의 이웃 크기를 찾기 위해서는 다양한 이웃 크기를 시도하면서 추천의 정확도를 최고로하는 이웃 크기를 실험을 통해서 찾아야 한다  

우선 10~60까지 10단위로 변화시키면서 보고, 그 다음에 더 세밀하게 1단위로 변화시키면서 결과를 보면 된

In [41]:
# 최적의 neighbor size 구하기
# train set으로 full matrix와 cosine similarity 구하기
rating_matrix = x_train.pivot_table(values='rating',index='user_id',columns='movie_id')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,index=rating_matrix.index,columns=rating_matrix.index)
for neighbor_size in [10,20,30,40,50,60]:
    print("Neighbor size = %d : RMSE = %.4f"%(neighbor_size,score(cf_knn,neighbor_size)))

Neighbor size = 10 : RMSE = 1.0257
Neighbor size = 20 : RMSE = 1.0125
Neighbor size = 30 : RMSE = 1.0102
Neighbor size = 40 : RMSE = 1.0104
Neighbor size = 50 : RMSE = 1.0107
Neighbor size = 60 : RMSE = 1.0115


In [42]:
# 1단위로 최적의 이웃 숫자 찾기
for neighbor_size in range(30,40):
    print("Neighbor size = %d : RMSE = %.4f"%(neighbor_size,score(cf_knn,neighbor_size)))

Neighbor size = 30 : RMSE = 1.0102
Neighbor size = 31 : RMSE = 1.0100
Neighbor size = 32 : RMSE = 1.0099
Neighbor size = 33 : RMSE = 1.0098
Neighbor size = 34 : RMSE = 1.0099
Neighbor size = 35 : RMSE = 1.0099
Neighbor size = 36 : RMSE = 1.0100
Neighbor size = 37 : RMSE = 1.0101
Neighbor size = 38 : RMSE = 1.0103
Neighbor size = 39 : RMSE = 1.0104


## 3.6 사용자의 평가경향을 고려한 CF

CF의 정확도를 더 개선시키는 방법 중의 하나는 사용자의 평가경향(user bias)을 고려해서 예측치를 조정하는 것이다.  
사용자에 따라서 쳥가를 저체적으로 높게 하는 사람이 있는 반면에 전체적으로 낮게 하는 사람도 있는 등 사람에 따라 평가경향이 다르다.  

<br>

1. 각 사용자의 평점평균을 구한다.

2. 각 아이템의 평점을 각 사용자의 평균에서의 차이(평점-해당 사용자의 전체 평점평균)로 변환
    - 편의상 평점과 평균의 차이를 평점편차로 부르기로

3. 평점편차를 사용해서 해당 사용자의 해당 아이템의 편차 예측값(평점편차의 예측값)을 구한다
    - 구체적으로 해당 사용자의 이웃을 구하고 이들 이웃의 해당 아이템에 대한 평점편차와 유사도를 가중평균한다

4. 이렇게 구한 편차 예측값은 평균에서의 차이를 의미하기 때문에 실제 예측값으로 변환하기 위해서 현 사용자의 평균에 이 편차 예측값을 더해준다.

5. 예측값을 구할 수 없는 경우에 지금까지는 3.0을 할당했는데, 이번에는 해당 사용자의 평점평균으로 대체한다.

In [46]:
# 모든 user의 rating 평균과 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis=1) # 유저별로 평균계산
rating_bias = (rating_matrix.T - rating_mean).T # 열이 user_id로 하기 위해 전치

def CF_knn_bias(user_id,movie_id,neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_bias[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        # neighbor size가 지정되지 않은 경우
        if neighbor_size ==0:
            prediction = np.dot(sim_scores,movie_ratings) / sim_scores.sum()
            prediction = prediction + rating_mean[user_id]
        else:
            if len(sim_scores)>1:
                neighbor_size = min(neighbor_size,len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores,movie_ratings) / sim_scores.sum()
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    
    return prediction

score(CF_knn_bias,30)

0.9433523871233872